# Partially Observed Data

### Pseudo-code

<pre>
<i><b>Expectation-Maximization</b></i> (
    &theta;<sup>(0)</sup> // Initial parameters
    D   // Data
  )
1    <b>for</b> each iteration t
2        M<sub>t</sub> &larr; E-step(&theta;<sup>(t)</sup>, D)
3        &theta;<sup>(t+1)</sup> &larr; M-step(M<sub>t</sub>)
4    <b>return</b> &theta;<sup>(T)</sup>

<i><b>E-step</b></i> (
    &theta;<sup>(t)</sup> // Current parameters
    D   // Data
  )
1    M<sub>t</sub> &larr; 0
2    <b>for</b> each data point d
3        <b>for</b> each node X
4            <b>for</b> each combination X=x, Pa(X)=y
5                M<sub>t</sub>[x, y] &larr; M<sub>t</sub>[x, y] + p(x, y|d, &theta;<sup>(t)</sup>)
6    <b>return</b> M<sub>t</sub>
    

<i><b>M-step</b></i> (
    M<sub>t</sub>  // Expected sufficient statistics
  )
1    <b>for</b> each node X
2        <b>for</b> each combination X=x, Pa(X)=y
3            &theta;<sup>(t+1)</sup>[x|y] &larr; M<sub>t</sub>[x, y] / M<sub>t</sub>[p]
4    <b>return</b> &#x03B8;<sup>(t+1)</sup>
</pre>

In [1]:
import numpy as np

from em import expectation_maximization, generate_data, print_tables, print_marginals

p(x) = [0.568 0.432]

p(y) = [0.294 0.706]


```
p(z|x=0, y=0) = {pz[0, 0]}

p(z|x=0, y=1) = {pz[0, 1]}

p(z|x=1, y=0) = {pz[1, 0]}

p(z|x=1, y=1) = {pz[1, 1]}
```

In [2]:
def e_step(qx, qy, qz, xs, ys, zs):
    Mx = np.zeros(2)
    My = np.zeros(2)
    Mz = np.zeros((2, 2, 2))  # Remember index order: Mz[x, y, z]
    for x, y, z,l in zip(xs, ys, zs,range(500)):
   
        """
        To do:  p(X|x, y, z) and add to Mx (The M θ (t) are the expected sufficient statistics)
                p(Y|x, y, z) and add to My
                p(Z, X, Y|x, y, z) and add to Mz.
            Remember to normalize p(.), i.e. each should sum to 1. 
                For example, if x, y and z are not None, we should have p(X=x) = 1, p(Y=y) = 1, p(Z=z, X=x, Y=y) = 1. 
        Naive solution (~45 lines of code): 
            x and y are None? ...
            x is None: ... (NB: p(X|Y=y, Z=z) = p(X, Y=y, Z=z) / p(Y=y, Z=z) != p(X))
            y is None: ...
            x and y are known: p(...) = 1
        Pythonic solution (<10 lines of code):
            Q = np.zeros((2, 2)) # Q(x, y) = p(Z=z, X=x, Y=y)
            # Q <- p(...)
            Mx += ...p(X|x, y, z)
            My += ...p(Y|x, y, z)
            Mz[:, :, z] += ... p(Z, X, Y|x, y, z)
        """
        # print(x,y,z)
        ####################################################
        # IN THE CASE THERE IS NO MISSING DATA
        ###################################################
        # if (x != None) and (y != None) and (z != None):
        #     Q = np.zeros((2,2))
        #     Q[x,y] = 1
        #     Mx +=  Q[:,y] #(qx/p_z_x_y)/sum(qx/p_z_x_y)
        #     My += Q[x] #(qy/p_z_x_y)/sum(qy/p_z_x_y)
        #     Mz[:,:,z] += Q # 
        #     continue 
        
        ###################################################
        # IN THE CASE THERE IS A MISSING DATA 
        ###################################################
        
        
        # if the data is missing, otherwise 
        # qx = p(x) , qy = p(y), qz = p(z|x,y)
        # 0           1            1    pz1 = p(x)p(y)p(z|x,y)  --> q = pz1/pz1+pz2
        # 0           1            0    pz2 = p(x)p(y)p(z|x,y)  --> q = pz2/pz1+pz2
        # 1           0            1    pz3 = p(x)p(y)p(z|x,y)  --> q = pz3/pz3+pz4
        # 1           0            0    pz4 = p(x)p(y)p(z|x,y)  --> q = pz4/pz3+pz4
        # 0           0            1    pz5
        # 0           0            0    pz6
        # 1           1            1    pz7
        # 1           1            0    pz8
        
        # we need to to compute the weights of each possible occurance and then add this 
        # if the x or y missing, they are indepenedent 
        if (x != None) and (y != None) and (z != None):
            Q = np.zeros((2,2))
            Q[x,y] = 1
            Mx +=  Q[:,y] #(qx/p_z_x_y)/sum(qx/p_z_x_y)
            My += Q[x] #(qy/p_z_x_y)/sum(qy/p_z_x_y)
            Mz[:,:,z] += Q # 
            
# not sure but this seems like what you wanted (as a pythonic solution) but it yields a lower probability and does not seem to be realyl correct (probably my fault), thus i went with less elegant way  
#         else:
            
#             Mx += qx
#             My += qy 
#             Q = qx*qy*qz
#             # My[y] += 1 #(qy/p_z_x_y)/sum(qy/p_z_x_y)
#             Mz += Q/ np.sum(Q)
        
        
        elif x == None and y == None: 
            Mx += qx
            My += qy 
            # My[y] += 1 #(qy/p_z_x_y)/sum(qy/p_z_x_y)
            Mz[:,:,z] += (qx *qy* qz[:,:,z])/ np.sum(qx *qy * qz[:,:,z])
            # print((qx *qy* qz[:,:,z])/ np.sum(qx *qy * qz[:,:,z]))
        # there is no situation where the x and z or y and z are missing, so i do not consider that 
        elif x == None:
            Mx += qx
            My[y] += 1 #(qy/p_z_x_y)/sum(qy/p_z_x_y)
            Mz[:,y,z] += (qx * qz[:,y,z])/ np.sum(qx * qz[:,y,z])
            # print((qx * qz[:,y,z])/ np.sum(qx * qz[:,y,z]))
        elif y == None:
            My += qy 
            Mx[x] += 1 #(qy/p_z_x_y)/sum(qy/p_z_x_y)
            Mz[x,:,z] += (qy * qz[x,:,z])/ np.sum(qy * qz[x,:,z])
        elif z == None: # and x and y are known 
            Q = np.zeros((2,2))
            Q[x,y] = 1
            Mx +=  Q[:,y] #(qx/p_z_x_y)/sum(qx/p_z_x_y)
            My += Q[x] 
            Mz[x,y] += qz[x,y] / np.sum(qz[x,y])
            
    # print(np.sum(Mz[0])-np.sum(Mx[0]))
    return Mx, My, Mz

In [3]:
def m_step(Mx, My, Mz):
    """
    Convert from sufficient statistics to parameters. What elemets should sum to one?
    """
    qx = Mx / sum(Mx)
    qy = My / sum(My)
    # np.reshape(np.sum(Mz,axis=1),(2,2,1))
    qz = Mz / np.reshape(np.sum(Mz,axis=2),(2,2,1))
    return qx, qy, qz

## Run

In [4]:
np.random.seed(2018)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7 ,0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 500
x, y, z = generate_data(px, py, pz, n_data,partially_observed=True)
# for i in range(500):
#     if y[i] == z[i] and z[i] == None:
#         print('fuck')
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

In [54]:
# for purpose of question 4
np.random.seed(2018)
diff = 0
for pxi in [1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0]:
    for pyi in [1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0]: 
        
        px = np.array([pxi, 1-pxi])
        py = np.array([pyi, 1-pyi])
        pz = np.array([[[0.2, 0.8], [0.1, 0.9]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
        n_data = 50
        x, y, z = generate_data(px, py, pz, n_data,partially_observed=True)
        # for i in range(500):
        #     if y[i] == z[i] and z[i] == None:
        #         print('fuck')
        n_iter = 10
        qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)
        if diff < sum(abs(qx-px)+abs(qy-py)):
            diff = sum(abs(qx-px)+abs(qy-py))
            print(pxi,pyi)
            print(sum(abs(qx-px)+abs(qy-py)))

1.0 1.0
0.03136673761821597
1.0 0.8
0.1090699263999996
1.0 0.6
0.16346402656000014
1.0 0.4
0.18821201762598272
1.0 0.2
0.24306409658556627
0.8 0.6
0.30121836827323156
0.8 0.4
0.40000000000000036
0.6 0.8
0.5803398255657937
0.4 0.1
1.0


In [5]:
print("Learnt parameters")
print("----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("--------------")
print_tables(px, py, pz)

Learnt parameters
----------------
p(x) = [0.62705125 0.37294875]
p(y) = [0.3168482 0.6831518]
p(z|x=0, y=0) = [0.25610379 0.74389621]
p(z|x=0, y=1) = [0.68720144 0.31279856]
p(z|x=1, y=0) = [0.88428318 0.11571682]
p(z|x=1, y=1) = [0.06647732 0.93352268]

True parameters
--------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


In [30]:
print("Learnt marginals")
print("----------------")
print_marginals(qx, qy, qz)
print()
print("True marginals")
print("--------------")
print_marginals(px, py, pz)

Learnt marginals
----------------
p(z|x=0) = [0.31683705 0.68316295]
p(z|x=1) = [0.3899365 0.6100635]
p(z|y=0) = [0.9199275 0.0800725]
p(z|y=1) = [0.05519159 0.94480841]

True marginals
--------------
p(z|x=0) = [0.55 0.45]
p(z|x=1) = [0.34 0.66]
p(z|y=0) = [0.9 0.1]
p(z|y=1) = [0.1 0.9]



# QUESTION 2 
Code above, reported values with seed 2018
```
p(x) = [0.586 0.414]

p(y) = [0.316 0.684]

p(z|x=0, y=0) = [0.20212766 0.79787234]

p(z|x=0, y=1) = [0.70351759 0.29648241]

p(z|x=1, y=0) = [0.890625 0.109375]

p(z|x=1, y=1) = [0.09090909 0.90909091]

True parameters
---------------

p(x) = [0.6 0.4]

p(y) = [0.3 0.7]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.7 0.3]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]

Learnt marginals
----------------


p(z|x=0) = [0.54507837 0.45492163]

p(z|x=1) = [0.34361932 0.65638068]

p(z|y=0) = [0.48716556 0.51283444]

p(z|y=1) = [0.44989767 0.55010233]


True marginals

--------------
p(z|x=0) = [0.55 0.45]


p(z|x=1) = [0.34 0.66]


p(z|y=0) = [0.48 0.52]


p(z|y=1) = [0.46 0.54]


```

# Question 3 
I had to uncomment the assertation, there was a slight difference in the sum, at the beginning 2-3 units, after 3 iterations 0.3 units. 
The weird thing is that it does not happen at $1^{st}$ iteration step but starts from the $2^{nd}$ one...

```assert np.isclose(np.sum(Mz[0]), Mx[0]), f"Mz[0] = {Mz[0]} should sum to Mx[0] = {Mx[0]}"```

there is also a slight difference in the results, which could be a mistake I did in the code (?)
Here are the resutls with seed = 1337

```
Learnt parameters

-----------------

p(x) = [0.56917956 0.43082044]

p(y) = [0.28526338 0.71473662]

p(z|x=0, y=0) = [0.28166043 0.71833957]

p(z|x=0, y=1) = [0.6009467 0.3990533]

p(z|x=1, y=0) = [0.91737932 0.08262068]

p(z|x=1, y=1) = [0.09256873 0.90743127]


True parameters

---------------

p(x) = [0.6 0.4]

p(y) = [0.3 0.7]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.7 0.3]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]
```

and with seed 2018, there the difference was quite high, up to 14 units.

```

Learnt parameters

-----------------

p(x) = [0.62705125 0.37294875]

p(y) = [0.3168482 0.6831518]

p(z|x=0, y=0) = [0.25610379 0.74389621]

p(z|x=0, y=1) = [0.68720144 0.31279856]

p(z|x=1, y=0) = [0.88428318 0.11571682]

p(z|x=1, y=1) = [0.06647732 0.93352268]


True parameters
--------------
p(x) = [0.6 0.4]

p(y) = [0.3 0.7]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.7 0.3]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]

```

I also iterate over some of the possible probability values and the highest absolute cummulative difference (0.45) between GT and the predicted values is for the GT probabilities 

px = (0.1, 0.9)
py = (0.8, 0.2)



# Question 4

Setting px to (0,1)
In this case differences are pretty high (a couple of %). Additionally, the issue which I reported in the previous question seems to repeat and the difference is really observable (~40 units). Nevertheless we are still able to predict the actual values quite well:

```
Learnt parameters

-----------------

p(x) = [3.67149593e-05 9.99963285e-01]

p(y) = [0.31614058 0.68385942]

p(z|x=0, y=0) = [0.15505673 0.84494327]

p(z|x=0, y=1) = [0.44084963 0.55915037]

p(z|x=1, y=0) = [0.8862824 0.1137176]

p(z|x=1, y=1) = [0.13982897 0.86017103]


True parameters

---------------

p(x) = [0 1]

p(y) = [0.3 0.7]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.7 0.3]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]
```

**10x less data**

The algorithm deals much better, suprisingly with similar scenario but less data giving more accurate results! It could be due to the fact that since there is less data, most of the examples have the values of the highly probable and it is easy to learn.

```
Learnt parameters

----------------

p(x) = [3.13910592e-05 9.99968609e-01]

p(y) = [0.61289614 0.38710386]

p(z|x=0, y=0) = [9.99999955e-01 4.50491652e-08]

p(z|x=0, y=1) = [9.99084993e-01 9.15006800e-04]

p(z|x=1, y=0) = [0.94457189 0.05542811]

p(z|x=1, y=1) = [0.09567451 0.90432549]


True parameters

--------------

p(x) = [0 1]

p(y) = [0.7 0.3]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.1 0.9]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]

```
Nevertheless, when it comes to the marginal, our model performed very poorly and there is a big difference between the GT and predicted distribution
 

```
Learnt marginals

----------------

p(z|x=0) = [0.31683705 0.68316295]

p(z|x=1) = [0.3899365 0.6100635]

p(z|y=0) = [0.9199275 0.0800725]

p(z|y=1) = [0.05519159 0.94480841]


True marginals

--------------

p(z|x=0) = [0.55 0.45]

p(z|x=1) = [0.34 0.66]

p(z|y=0) = [0.9 0.1]

p(z|y=1) = [0.1 0.9]
```

# Question 5 
With ```never_coobserved``` data flagged as true, our algorithm deal pretty well. We can see that the prediceted values are very close to the actual values
```
Learnt parameters

-----------------

p(x) = [0.586 0.414]

p(y) = [0.316 0.684]

p(z|x=0, y=0) = [0.20212766 0.79787234]

p(z|x=0, y=1) = [0.70351759 0.29648241]

p(z|x=1, y=0) = [0.890625 0.109375]

p(z|x=1, y=1) = [0.09090909 0.90909091]


True parameters

---------------

p(x) = [0.6 0.4]

p(y) = [0.3 0.7]

p(z|x=0, y=0) = [0.2 0.8]

p(z|x=0, y=1) = [0.7 0.3]

p(z|x=1, y=0) = [0.9 0.1]

p(z|x=1, y=1) = [0.1 0.9]
```
Naivly, we coudl say that it should be harder to learn such a distribution, however since it is ```x``` or ```y``` missing, it may be a big easier due to the fact that ```z``` is dependent on these variables. Due to this fact it it seems that data is missing but **not** at random (maybe MNAR)? This dependency also explains why mariginal distributions are even closer to the ```true``` ones. We can see them printed below:
```
Learnt marginals

----------------

p(z|x=0) = [0.54507837 0.45492163]

p(z|x=1) = [0.34361932 0.65638068]

p(z|y=0) = [0.48716556 0.51283444]

p(z|y=1) = [0.44989767 0.55010233]


True marginals

--------------

p(z|x=0) = [0.55 0.45]

p(z|x=1) = [0.34 0.66]

p(z|y=0) = [0.48 0.52]

p(z|y=1) = [0.46 0.54]
```

Sorry, I m not sure what you mean about "Does the MAR tell the whole store :)". I hope I provided enough details in my other answers!